[1] 모듈 로딩 및 데이터 준비

In [2]:
# %pip install scikit-learn

In [73]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',None) # 전체 열 출력하기
pd.set_option('display.max_rows',None) # 전체 행 출력하기
from sklearn.preprocessing import LabelEncoder

[2] 데이터 로딩 및 확인

In [4]:
df = pd.read_excel('../DATA/fire_final.xlsx.')
df.head()
# encoding='cp949'

,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,진화종료시간_년,진화종료시간_월,진화종료시간_일,진화종료시간_시간,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_구분,발생원인_세부원인,발생원인_기타,피해면적_합계
0,2024,12,31,14:15,화,2025,1,1,10:32,대구,대구,군위,삼국유사,화북,기,기타(직접입력),조사중,14.50
1,2024,12,30,15:00,월,2024,12,30,16:45,경기,경기,양평,강하,왕창,기,기타(직접입력),재 취급 부주의,0.06
2,2024,12,30,13:08,월,2024,12,30,15:57,경기,경기,파주,NaN,금릉,담,기타(직접입력),담뱃불 실화,0.01
3,2024,12,29,14:10,일,2024,12,29,15:00,강원,강원,강릉,NaN,운정,쓰,기타(직접입력),쓰레기 소각,0.01
4,2024,12,28,09:26,토,2024,12,28,12:00,강원,강원,양양,현북,도,기,기타(직접입력),원인미상,0.20


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5455 entries, 0 to 5454
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   발생일시_년     5455 non-null   int64  
 1   발생일시_월     5455 non-null   int64  
 2   발생일시_일     5455 non-null   int64  
 3   발생일시_시간    5455 non-null   object 
 4   발생일시_요일    5455 non-null   object 
 5   진화종료시간_년   5455 non-null   int64  
 6   진화종료시간_월   5455 non-null   int64  
 7   진화종료시간_일   5455 non-null   int64  
 8   진화종료시간_시간  5455 non-null   object 
 9   발생장소_관서    5454 non-null   object 
 10  발생장소_시도    5455 non-null   object 
 11  발생장소_시군구   5432 non-null   object 
 12  발생장소_읍면    4919 non-null   object 
 13  발생장소_동리    4934 non-null   object 
 14  발생원인_구분    4514 non-null   object 
 15  발생원인_세부원인  5455 non-null   object 
 16  발생원인_기타    5455 non-null   object 
 17  피해면적_합계    5455 non-null   float64
dtypes: float64(1), int64(6), object(11)
memory usage: 767.2+ KB


In [6]:
print(df['발생원인_기타'].value_counts())

발생원인_기타
입산자 실화         1779
쓰레기 소각         1057
원인미상            769
담뱃불 실화          413
건축물 화재          236
논, 밭두렁 소각       223
사업장 실화          153
화기물 취급 실화       128
불씨취급부주의         105
전기 화재            77
기타               67
재 취급 부주의         65
방화               62
공장화재비화           45
용접작업실화           45
조사중              43
공사장실화            36
군부대 훈련중 부주의      26
차량 화재            25
낙뢰               22
불장난              16
무속행위             16
재발화              15
낚시꾼              10
취사행위              9
과열 화재             5
자연발화              4
불법소각              2
정신지체자 실화          2
Name: count, dtype: int64


In [7]:
count_with_spaces = df['발생원인_기타'].str.contains(' ', na=False).sum()
print(f"공백이 포함된 데이터의 개수: {count_with_spaces}")

공백이 포함된 데이터의 개수: 4189


In [8]:
df['발생원인_기타'] = df['발생원인_기타'].str.replace(' ', '', regex=False)

count_with_spaces = df['발생원인_기타'].str.contains(' ', na=False).sum()
print(f"공백이 포함된 데이터의 개수: {count_with_spaces}")

공백이 포함된 데이터의 개수: 0


In [9]:
# 공백 제거 후, 발생원인_기타 건수 확인
print(df['발생원인_기타'].value_counts())

발생원인_기타
입산자실화        1779
쓰레기소각        1057
원인미상          769
담뱃불실화         413
건축물화재         236
논,밭두렁소각       223
사업장실화         153
화기물취급실화       128
불씨취급부주의       105
전기화재           77
기타             67
재취급부주의         65
방화             62
공장화재비화         45
용접작업실화         45
조사중            43
공사장실화          36
군부대훈련중부주의      26
차량화재           25
낙뢰             22
불장난            16
무속행위           16
재발화            15
낚시꾼            10
취사행위            9
과열화재            5
자연발화            4
불법소각            2
정신지체자실화         2
Name: count, dtype: int64


---

In [10]:
# 계절별로 월을 분류하는 함수
def get_season(month):
    if month in [3, 4, 5]:
        return '봄'
    elif month in [6, 7, 8]:
        return '여름'
    elif month in [9, 10, 11]:
        return '가을'
    else:
        return '겨울'

# '발생일시_월'을 이용해 '계절' 칼럼을 생성
df['계절'] = df['발생일시_월'].apply(get_season)

# 계절별 산불 발생 빈도 계산
season_count = df['계절'].value_counts()

print(season_count)

계절
봄     3028
겨울    1458
여름     503
가을     466
Name: count, dtype: int64


In [11]:
df.head()

,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,진화종료시간_년,진화종료시간_월,진화종료시간_일,진화종료시간_시간,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_구분,발생원인_세부원인,발생원인_기타,피해면적_합계,계절
0,2024,12,31,14:15,화,2025,1,1,10:32,대구,대구,군위,삼국유사,화북,기,기타(직접입력),조사중,14.50,겨울
1,2024,12,30,15:00,월,2024,12,30,16:45,경기,경기,양평,강하,왕창,기,기타(직접입력),재취급부주의,0.06,겨울
2,2024,12,30,13:08,월,2024,12,30,15:57,경기,경기,파주,NaN,금릉,담,기타(직접입력),담뱃불실화,0.01,겨울
3,2024,12,29,14:10,일,2024,12,29,15:00,강원,강원,강릉,NaN,운정,쓰,기타(직접입력),쓰레기소각,0.01,겨울
4,2024,12,28,09:26,토,2024,12,28,12:00,강원,강원,양양,현북,도,기,기타(직접입력),원인미상,0.20,겨울


In [12]:
# "발생원인_기타" 값이 "입산자실화"인 데이터 필터링
filtered_df = df[df['발생원인_기타'] == '입산자실화']

# 계절별 "입산자실화" 발생 건수 합산
seasonal_counts = filtered_df.groupby('계절')['발생원인_기타'].value_counts()
seasonal_counts

계절  발생원인_기타
가을  입산자실화      195
겨울  입산자실화      405
봄   입산자실화      977
여름  입산자실화      202
Name: count, dtype: int64

---

[3] 데이터 확인

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5455 entries, 0 to 5454
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   발생일시_년     5455 non-null   int64  
 1   발생일시_월     5455 non-null   int64  
 2   발생일시_일     5455 non-null   int64  
 3   발생일시_시간    5455 non-null   object 
 4   발생일시_요일    5455 non-null   object 
 5   진화종료시간_년   5455 non-null   int64  
 6   진화종료시간_월   5455 non-null   int64  
 7   진화종료시간_일   5455 non-null   int64  
 8   진화종료시간_시간  5455 non-null   object 
 9   발생장소_관서    5454 non-null   object 
 10  발생장소_시도    5455 non-null   object 
 11  발생장소_시군구   5432 non-null   object 
 12  발생장소_읍면    4919 non-null   object 
 13  발생장소_동리    4934 non-null   object 
 14  발생원인_구분    4514 non-null   object 
 15  발생원인_세부원인  5455 non-null   object 
 16  발생원인_기타    5455 non-null   object 
 17  피해면적_합계    5455 non-null   float64
 18  계절         5455 non-null   object 
dtypes: float64(1), int64(6), object(12)
memory usage

결측치 확인

In [14]:
df.isnull().sum()
# df = df.dropna()

발생일시_년         0
발생일시_월         0
발생일시_일         0
발생일시_시간        0
발생일시_요일        0
진화종료시간_년       0
진화종료시간_월       0
진화종료시간_일       0
진화종료시간_시간      0
발생장소_관서        1
발생장소_시도        0
발생장소_시군구      23
발생장소_읍면      536
발생장소_동리      521
발생원인_구분      941
발생원인_세부원인      0
발생원인_기타        0
피해면적_합계        0
계절             0
dtype: int64

In [15]:
## '발생원인_구분'(null이 존재하는)의 고유값 확인
df['발생원인_구분'].unique()

array(['기', '담', '쓰', nan, '입'], dtype=object)

In [16]:
df['발생원인_세부원인'].unique()

array(['기타(직접입력)', '작업장실화', '주택화재비화', '방화', '농산폐기물소각', '담뱃불실화', '성묘객실화',
       '논밭두렁소각', '쓰레기소각', '낙뢰', '어린이불장난', '무속행위'], dtype=object)

In [17]:
df['발생원인_기타'].unique()

array(['조사중', '재취급부주의', '담뱃불실화', '쓰레기소각', '원인미상', '건축물화재', '입산자실화',
       '화기물취급실화', '전기화재', '차량화재', '사업장실화', '불씨취급부주의', '기타', '공장화재비화',
       '방화', '논,밭두렁소각', '낚시꾼', '용접작업실화', '불장난', '군부대훈련중부주의', '공사장실화',
       '낙뢰', '취사행위', '과열화재', '무속행위', '자연발화', '재발화', '불법소각', '정신지체자실화'],
      dtype=object)

In [18]:
## '발생원인_구분'이 결측치인 것 추출
cause_gu = df[df['발생원인_구분'].isnull()]
cause_gu

,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,진화종료시간_년,진화종료시간_월,진화종료시간_일,진화종료시간_시간,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_구분,발생원인_세부원인,발생원인_기타,피해면적_합계,계절
8,2024,12,24,18:22,화,2024,12,24,22:00,경북,경북,구미,산동,백현,NaN,기타(직접입력),건축물화재,0.15,겨울
9,2024,12,23,06:48,월,2024,12,23,08:25,강원,강원,고성,거진,초계,NaN,기타(직접입력),건축물화재,0.16,겨울
39,2024,11,16,05:13,토,2024,11,16,08:59,경기,경기,광주,NaN,회덕,NaN,기타(직접입력),사업장실화,1.00,가을
55,2024,9,5,13:51,목,2024,9,5,16:10,경북,경북,안동,녹전,매정,NaN,기타(직접입력),쓰레기소각,0.05,가을
78,2024,6,14,12:59,금,2024,6,14,15:37,충남,충남,홍성,장곡,행정,NaN,주택화재비화,건축물화재,0.03,여름
88,2024,6,5,15:37,수,2024,6,5,18:00,경기,경기,광주,초월,신월,NaN,기타(직접입력),건축물화재,0.03,여름
91,2024,6,1,15:22,토,2024,6,1,18:35,경기,경기,파주,탄현,축현,NaN,주택화재비화,공장화재비화,0.99,여름
97,2024,5,24,13:27,금,2024,5,24,17:20,전남,전남,화순,사평,검산,NaN,기타(직접입력),"논,밭두렁소각",0.50,봄
103,2024,5,4,11:49,토,2024,5,4,12:27,인천,인천,강화,양도,삼흥,NaN,기타(직접입력),"논,밭두렁소각",0.06,봄
112,2024,4,27,13:51,토,2024,4,27,14:14,충남,충남,금산,남일,마장,NaN,기타(직접입력),"논,밭두렁소각",0.03,봄


In [19]:
## '발생원인_구분'이 null인 것 중 '발생원인_세부원인' 확인
cause_gu['발생원인_세부원인'].unique()

array(['기타(직접입력)', '주택화재비화', '농산폐기물소각', '논밭두렁소각'], dtype=object)

In [20]:
df['발생원인_구분'].unique()

array(['기', '담', '쓰', nan, '입'], dtype=object)

In [21]:
df[df['발생원인_구분'] == '입']

,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,진화종료시간_년,진화종료시간_월,진화종료시간_일,진화종료시간_시간,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_구분,발생원인_세부원인,발생원인_기타,피해면적_합계,계절
11,2024,12,19,22:03,목,2024,12,20,10:00,경남,경남,남해,미조,미조,입,기타(직접입력),입산자실화,0.70,겨울
17,2024,12,15,12:54,일,2024,12,15,17:10,경북,경북,봉화,법전,법전,입,기타(직접입력),입산자실화,1.00,겨울
21,2024,12,12,12:47,목,2024,12,13,09:50,울산,울산,울주,범서,척과,입,기타(직접입력),입산자실화,2.20,겨울
42,2024,11,12,14:58,화,2024,11,12,16:09,북부,경기,양평,강상,세월,입,기타(직접입력),입산자실화,0.10,가을
59,2024,8,18,11:40,일,2024,8,18,16:00,울산,울산,울주,범서,서사,입,기타(직접입력),입산자실화,0.20,여름
66,2024,6,21,15:57,금,2024,6,21,20:00,전북,전북,남원,산내,백일,입,기타(직접입력),입산자실화,0.33,여름
87,2024,6,6,21:42,목,2024,6,6,21:57,북부,서울,중,NaN,장충2가,입,기타(직접입력),입산자실화,0.01,여름
96,2024,5,24,16:36,금,2024,5,24,19:10,강원,강원,삼척,원덕,이천,입,기타(직접입력),담뱃불실화,0.08,봄
100,2024,5,19,16:18,일,2024,5,19,17:30,경기,경기,안산 상록,NaN,건건,입,기타(직접입력),입산자실화,0.13,봄
102,2024,5,18,15:38,토,2024,5,18,16:07,대구,대구,남,NaN,대명,입,기타(직접입력),입산자실화,0.01,봄


---

## 기간별(연도별 / 계절 / 월 / 시간) 산불 빈도

In [22]:
df.head()

,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,진화종료시간_년,진화종료시간_월,진화종료시간_일,진화종료시간_시간,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_구분,발생원인_세부원인,발생원인_기타,피해면적_합계,계절
0,2024,12,31,14:15,화,2025,1,1,10:32,대구,대구,군위,삼국유사,화북,기,기타(직접입력),조사중,14.50,겨울
1,2024,12,30,15:00,월,2024,12,30,16:45,경기,경기,양평,강하,왕창,기,기타(직접입력),재취급부주의,0.06,겨울
2,2024,12,30,13:08,월,2024,12,30,15:57,경기,경기,파주,NaN,금릉,담,기타(직접입력),담뱃불실화,0.01,겨울
3,2024,12,29,14:10,일,2024,12,29,15:00,강원,강원,강릉,NaN,운정,쓰,기타(직접입력),쓰레기소각,0.01,겨울
4,2024,12,28,09:26,토,2024,12,28,12:00,강원,강원,양양,현북,도,기,기타(직접입력),원인미상,0.20,겨울


### 데이터 타입 확인

In [23]:
df.head(3)

,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,진화종료시간_년,진화종료시간_월,진화종료시간_일,진화종료시간_시간,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_구분,발생원인_세부원인,발생원인_기타,피해면적_합계,계절
0,2024,12,31,14:15,화,2025,1,1,10:32,대구,대구,군위,삼국유사,화북,기,기타(직접입력),조사중,14.50,겨울
1,2024,12,30,15:00,월,2024,12,30,16:45,경기,경기,양평,강하,왕창,기,기타(직접입력),재취급부주의,0.06,겨울
2,2024,12,30,13:08,월,2024,12,30,15:57,경기,경기,파주,NaN,금릉,담,기타(직접입력),담뱃불실화,0.01,겨울


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5455 entries, 0 to 5454
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   발생일시_년     5455 non-null   int64  
 1   발생일시_월     5455 non-null   int64  
 2   발생일시_일     5455 non-null   int64  
 3   발생일시_시간    5455 non-null   object 
 4   발생일시_요일    5455 non-null   object 
 5   진화종료시간_년   5455 non-null   int64  
 6   진화종료시간_월   5455 non-null   int64  
 7   진화종료시간_일   5455 non-null   int64  
 8   진화종료시간_시간  5455 non-null   object 
 9   발생장소_관서    5454 non-null   object 
 10  발생장소_시도    5455 non-null   object 
 11  발생장소_시군구   5432 non-null   object 
 12  발생장소_읍면    4919 non-null   object 
 13  발생장소_동리    4934 non-null   object 
 14  발생원인_구분    4514 non-null   object 
 15  발생원인_세부원인  5455 non-null   object 
 16  발생원인_기타    5455 non-null   object 
 17  피해면적_합계    5455 non-null   float64
 18  계절         5455 non-null   object 
dtypes: float64(1), int64(6), object(12)
memory usage

In [25]:
# 날짜는 동일해서 시간 계산 시 날짜 무시하고 칼럼 사용 예정
df['발생일시_시간'].unique()

array(['14:15', '15:00', '13:08', ..., '21:32', '00:26', '01:54'],
      dtype=object)

##### 시간 형변환

In [26]:
for col in df[['발생일시_시간', '진화종료시간_시간']]:
    for idx in df.index:
        df.loc[idx, col] = df.loc[idx, col][:2]

In [27]:
df['진화종료시간_시간'] = df['진화종료시간_시간'].astype('int')
df['발생일시_시간'] = df['발생일시_시간'].astype('int')
df.dtypes

발생일시_년         int64
발생일시_월         int64
발생일시_일         int64
발생일시_시간        int64
발생일시_요일       object
진화종료시간_년       int64
진화종료시간_월       int64
진화종료시간_일       int64
진화종료시간_시간      int64
발생장소_관서       object
발생장소_시도       object
발생장소_시군구      object
발생장소_읍면       object
발생장소_동리       object
발생원인_구분       object
발생원인_세부원인     object
발생원인_기타       object
피해면적_합계      float64
계절            object
dtype: object

In [28]:
df.head(1)

,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,진화종료시간_년,진화종료시간_월,진화종료시간_일,진화종료시간_시간,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_구분,발생원인_세부원인,발생원인_기타,피해면적_합계,계절
0,2024,12,31,14,화,2025,1,1,10,대구,대구,군위,삼국유사,화북,기,기타(직접입력),조사중,14.5,겨울


### 연도별 산불 빈도 확인

In [29]:
# 연도별 산불 빈도
df['발생일시_년'].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015])

In [30]:
print('연도별 산불 발생 건수')
df['발생일시_년'].value_counts()

연도별 산불 발생 건수


발생일시_년
2022    756
2017    692
2019    653
2015    623
2020    620
2023    596
2018    496
2016    391
2021    349
2024    279
Name: count, dtype: int64

---

### 계절별 산불 빈도 확인

#### 계절별 데이터 추출

In [31]:
# 계절별로 월을 분류하는 함수
def get_season(month):
    if month in [3, 4, 5]:
        return '봄'
    elif month in [6, 7, 8]:
        return '여름'
    elif month in [9, 10, 11]:
        return '가을'
    else:
        return '겨울'

# '발생일시_월'을 이용해 '계절' 칼럼을 생성
df['계절'] = df['발생일시_월'].apply(get_season)

# 계절별 산불 발생 빈도 계산
season_count = df['계절'].value_counts()

print(season_count)

계절
봄     3028
겨울    1458
여름     503
가을     466
Name: count, dtype: int64


- 봄에 산불이 가장 많이 발생하는 것으로 확인

---

#### 계절별 월별 산불 발생 건수

##### 계절별 데이터프레임 생성

In [32]:
month_spring = df[(df['발생일시_월'] == 3) | (df['발생일시_월'] == 4) | (df['발생일시_월'] == 5)]
month_spring.shape

(3028, 19)

In [33]:
month_summer = df[(df['발생일시_월'] == 6) | (df['발생일시_월'] == 7) | (df['발생일시_월'] == 8)]
month_summer.shape

(503, 19)

In [34]:
month_fall = df[(df['발생일시_월'] == 9) | (df['발생일시_월'] == 10) | (df['발생일시_월'] == 11)]
month_fall.shape

(466, 19)

In [35]:
month_winter = df[(df['발생일시_월'] == 12) | (df['발생일시_월'] == 1) | (df['발생일시_월'] == 2)]
month_winter.shape

(1458, 19)

In [36]:
# 4계절 추츨 확인
print(3028 + 503 + 466 +1458)

5455


In [37]:
print('봄의 월별 산불 발생 건수')
month_spring['발생일시_월'].value_counts()
# 3 > 4 > 5 월 순으로 산불이 가장 많이 발생함.

봄의 월별 산불 발생 건수


발생일시_월
3    1384
4    1130
5     514
Name: count, dtype: int64

In [38]:
print('여름의 월별 산불 발생 건수')
month_summer['발생일시_월'].value_counts()
# 6 > 8 > 7 월 순으로 산불이 가장 많이 발생함.

여름의 월별 산불 발생 건수


발생일시_월
6    368
8     87
7     48
Name: count, dtype: int64

In [39]:
print('가을의 월별 산불 발생 건수')
month_fall['발생일시_월'].value_counts()
# 11 > 10 > 9 월 순으로 산불이 가장 많이 발생함.

가을의 월별 산불 발생 건수


발생일시_월
11    259
10    138
9      69
Name: count, dtype: int64

In [40]:
print('겨울의 월별 산불 발생 건수')
month_winter['발생일시_월'].value_counts()
# 2 > 1 > 12 월 순으로 산불이 가장 많이 발생함.

겨울의 월별 산불 발생 건수


발생일시_월
2     741
1     425
12    292
Name: count, dtype: int64

### 월별 산불 빈도 확인

In [41]:
# 데이터 확인
df.head(3)

,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,진화종료시간_년,진화종료시간_월,진화종료시간_일,진화종료시간_시간,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_구분,발생원인_세부원인,발생원인_기타,피해면적_합계,계절
0,2024,12,31,14,화,2025,1,1,10,대구,대구,군위,삼국유사,화북,기,기타(직접입력),조사중,14.50,겨울
1,2024,12,30,15,월,2024,12,30,16,경기,경기,양평,강하,왕창,기,기타(직접입력),재취급부주의,0.06,겨울
2,2024,12,30,13,월,2024,12,30,15,경기,경기,파주,NaN,금릉,담,기타(직접입력),담뱃불실화,0.01,겨울


In [42]:
df['발생일시_월'].unique()

array([12, 11, 10,  9,  8,  6,  5,  4,  3,  2,  1,  7])

In [43]:
print('월별 산불 발생 건수')
df['발생일시_월'].value_counts()    # 3월 > 4월 > 2월순

월별 산불 발생 건수


발생일시_월
3     1384
4     1130
2      741
5      514
1      425
6      368
12     292
11     259
10     138
8       87
9       69
7       48
Name: count, dtype: int64

### 시간별 산불 빈도 확인

In [44]:
df.head()

,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,진화종료시간_년,진화종료시간_월,진화종료시간_일,진화종료시간_시간,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_구분,발생원인_세부원인,발생원인_기타,피해면적_합계,계절
0,2024,12,31,14,화,2025,1,1,10,대구,대구,군위,삼국유사,화북,기,기타(직접입력),조사중,14.50,겨울
1,2024,12,30,15,월,2024,12,30,16,경기,경기,양평,강하,왕창,기,기타(직접입력),재취급부주의,0.06,겨울
2,2024,12,30,13,월,2024,12,30,15,경기,경기,파주,NaN,금릉,담,기타(직접입력),담뱃불실화,0.01,겨울
3,2024,12,29,14,일,2024,12,29,15,강원,강원,강릉,NaN,운정,쓰,기타(직접입력),쓰레기소각,0.01,겨울
4,2024,12,28,9,토,2024,12,28,12,강원,강원,양양,현북,도,기,기타(직접입력),원인미상,0.20,겨울


In [45]:
df['발생일시_시간'].unique()

array([14, 15, 13,  9, 22, 12, 18,  6, 11,  5, 17, 16, 10, 19,  8,  1,  4,
        3, 21,  0, 20, 23,  2,  7])

In [46]:
print('시간별 산불 발생 건수')
df['발생일시_시간'].value_counts()

시간별 산불 발생 건수


발생일시_시간
14    845
13    806
15    692
12    639
11    438
16    427
17    261
10    247
18    190
19    147
9     111
21     89
20     83
5      59
22     58
23     54
0      49
8      49
4      42
3      41
6      38
7      34
2      29
1      27
Name: count, dtype: int64

In [47]:
# bins = pd.IntervalIndex.from_tuples([(0,6), (6,12), (12,18),(18,24)])/
# df['시간대'] = pd.cut(df['발생일시_시간'], bins=bins, labels=['새벽', '오전', '오후', '저녁'] )
bins = [0, 6, 12, 18, 24]
labels = ['새벽', '오전', '오후', '저녁']
df['시간대'] = pd.cut(df['발생일시_시간'], bins=bins, labels=labels )

df.head(2)

,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,진화종료시간_년,진화종료시간_월,진화종료시간_일,진화종료시간_시간,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_구분,발생원인_세부원인,발생원인_기타,피해면적_합계,계절,시간대
0,2024,12,31,14,화,2025,1,1,10,대구,대구,군위,삼국유사,화북,기,기타(직접입력),조사중,14.50,겨울,오후
1,2024,12,30,15,월,2024,12,30,16,경기,경기,양평,강하,왕창,기,기타(직접입력),재취급부주의,0.06,겨울,오후


In [48]:
print('시간대별 산불 발생 건수')
df['시간대'].value_counts()

시간대별 산불 발생 건수


시간대
오후    3221
오전    1518
저녁     431
새벽     236
Name: count, dtype: int64

---

# 상관관계 분석

In [74]:
df.head(1)

,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,진화종료시간_년,진화종료시간_월,진화종료시간_일,진화종료시간_시간,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_구분,발생원인_세부원인,발생원인_기타,피해면적_합계,계절,시간대
0,2024,12,31,14,6,2025,1,1,10,대구,대구,군위,삼국유사,화북,기,기타(직접입력),조사중,14.5,겨울,오후


In [75]:
df_corr = df[['발생일시_년', '발생일시_월', '발생일시_일', '발생일시_시간', '발생일시_요일', '발생장소_관서', '발생장소_시도', '발생장소_시군구', '발생장소_읍면', '발생장소_동리', '발생원인_기타', '피해면적_합계', '계절']]
df_corr.head(2)

,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_기타,피해면적_합계,계절
0,2024,12,31,14,6,대구,대구,군위,삼국유사,화북,조사중,14.50,겨울
1,2024,12,30,15,3,경기,경기,양평,강하,왕창,재취급부주의,0.06,겨울


In [77]:
encoder = LabelEncoder()
df_corr['발생일시_요일'] = encoder.fit_transform(df_corr['발생일시_요일'])
df_corr['발생일시_요일'] = encoder.fit_transform(df_corr['발생일시_요일'])
df_corr['발생일시_요일'] = encoder.fit_transform(df_corr['발생일시_요일'])
df_corr['발생일시_요일'] = encoder.fit_transform(df_corr['발생일시_요일'])
df_corr['발생일시_요일'] = encoder.fit_transform(df_corr['발생일시_요일'])
df_corr['발생일시_요일'] = encoder.fit_transform(df_corr['발생일시_요일'])
df_corr['발생일시_요일'] = encoder.fit_transform(df_corr['발생일시_요일'])
df_corr['발생일시_요일'] = encoder.fit_transform(df_corr['발생일시_요일'])

C:\Users\KDP-519\AppData\Local\Temp\ipykernel_7576\1311511588.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['발생일시_요일'] = encoder.fit_transform(df_corr['발생일시_요일'])
C:\Users\KDP-519\AppData\Local\Temp\ipykernel_7576\1311511588.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['발생일시_요일'] = encoder.fit_transform(df_corr['발생일시_요일'])
C:\Users\KDP-519\AppData\Local\Temp\ipykernel_7576\1311511588.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [82]:
for col in df_corr.select_dtypes(include="object").columns:
    df_corr[col] = encoder.fit_transform(df_corr[col])
    
corrlation = df_corr.corr()
corrlation

C:\Users\KDP-519\AppData\Local\Temp\ipykernel_7576\3658879022.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr[col] = encoder.fit_transform(df_corr[col])
C:\Users\KDP-519\AppData\Local\Temp\ipykernel_7576\3658879022.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr[col] = encoder.fit_transform(df_corr[col])
C:\Users\KDP-519\AppData\Local\Temp\ipykernel_7576\3658879022.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_기타,피해면적_합계,계절
발생일시_년,1.000000,-0.020431,-0.057320,-0.011982,-0.004566,0.052848,0.064421,-0.014094,0.104430,-0.119571,-0.021539,0.022639,-0.113199
발생일시_월,-0.020431,1.000000,0.000325,-0.016152,-0.022685,0.008029,-0.003070,-0.029362,0.006295,-0.019369,0.031791,-0.012334,-0.267036
발생일시_일,-0.057320,0.000325,1.000000,0.014799,0.003306,0.014013,0.018162,0.001894,0.007628,-0.004753,0.017490,-0.025838,-0.133545
발생일시_시간,-0.011982,-0.016152,0.014799,1.000000,0.019473,0.009938,0.006647,0.004214,-0.012520,-0.022750,-0.019780,-0.017542,-0.008102
발생일시_요일,-0.004566,-0.022685,0.003306,0.019473,1.000000,0.006128,0.009543,-0.014802,0.027810,0.015417,0.026231,-0.018688,-0.020161
발생장소_관서,0.052848,0.008029,0.014013,0.009938,0.006128,1.000000,0.961235,-0.076630,-0.029118,0.015028,0.053994,-0.013467,0.029849
발생장소_시도,0.064421,-0.003070,0.018162,0.006647,0.009543,0.961235,1.000000,-0.105011,-0.024037,0.019619,0.046093,-0.010722,0.022861
발생장소_시군구,-0.014094,-0.029362,0.001894,0.004214,-0.014802,-0.076630,-0.105011,1.000000,-0.006947,-0.054016,-0.014539,-0.001044,0.037028
발생장소_읍면,0.104430,0.006295,0.007628,-0.012520,0.027810,-0.029118,-0.024037,-0.006947,1.000000,-0.029879,0.004649,-0.005061,-0.020281
발생장소_동리,-0.119571,-0.019369,-0.004753,-0.022750,0.015417,0.015028,0.019619,-0.054016,-0.029879,1.000000,0.021571,-0.014486,0.013780


```
0.00 ~ ±0.10	            거의 없음 (무상관)	             두 변수 간 상관이 거의 없음
±0.10 ~ ±0.30	            약한 상관관계	                약한 선형 관계
±0.30 ~ ±0.50	            중간 정도의 상관관계	         비교적 뚜렷한 관계
±0.50 ~ ±0.70	            강한 상관관계	                변수 간 강한 선형 관계
±0.70 ~ ±1.00	            매우 강한 상관관계	            매우 강한 선형 관계
```

- 눈에 띄게 상관관계를 띄는 것은 없다.

---

In [52]:
df.head(1)

,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,진화종료시간_년,진화종료시간_월,진화종료시간_일,진화종료시간_시간,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_구분,발생원인_세부원인,발생원인_기타,피해면적_합계,계절,시간대
0,2024,12,31,14,6,2025,1,1,10,대구,대구,군위,삼국유사,화북,기,기타(직접입력),조사중,14.5,겨울,오후


#### 연도별 발생장소(시도) 산불 발생 건수

In [53]:
year_location = df.groupby('발생일시_년')['발생장소_시도'].value_counts()
most_freq_location = year_location.groupby('발생일시_년').idxmax()
year_location = year_location[most_freq_location]
year_location

발생일시_년  발생장소_시도
2015    경기         150
2016    경기         113
2017    경기         161
2018    경북         100
2019    경기         172
2020    경기         213
2021    경기          74
2022    경기         154
2023    경기         107
2024    경기          86
Name: count, dtype: int64

#### 계절별 발생장소(시도) 산불 발생 건수

In [54]:
season_location = df.groupby('계절')['발생장소_시도'].value_counts()
most_freq_location = season_location.groupby('계절').idxmax()
season_location = season_location[most_freq_location]
season_location

계절  발생장소_시도
가을  경기          94
겨울  경북         276
봄   경기         835
여름  경북          98
Name: count, dtype: int64

#### 월별 발생장소(시도) 산불 발생 건수

In [56]:
df.head(1)

,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,진화종료시간_년,진화종료시간_월,진화종료시간_일,진화종료시간_시간,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_구분,발생원인_세부원인,발생원인_기타,피해면적_합계,계절,시간대
0,2024,12,31,14,6,2025,1,1,10,대구,대구,군위,삼국유사,화북,기,기타(직접입력),조사중,14.5,겨울,오후


In [57]:
month_location = df.groupby('발생일시_월')['발생장소_시도'].value_counts()
most_freq_location = month_location.groupby('발생일시_월').idxmax()
month_location = month_location[most_freq_location]
month_location

발생일시_월  발생장소_시도
1       경기          86
2       경기         147
3       경기         394
4       경기         334
5       강원         128
6       경기          76
7       경북          10
8       경북          14
9       충남          13
10      경북          29
11      경기          63
12      경북          57
Name: count, dtype: int64

#### 연도별 발생원인(기타) 산불 발생 건수

In [60]:
year_cause = df.groupby('발생일시_년')['발생원인_기타'].value_counts()
most_freq_cause = year_cause.groupby('발생일시_년').idxmax()
year_cause = year_cause[most_freq_cause]
year_cause

발생일시_년  발생원인_기타
2015    입산자실화      197
2016    쓰레기소각      127
2017    입산자실화      266
2018    입산자실화      169
2019    입산자실화      185
2020    입산자실화      222
2021    입산자실화      137
2022    입산자실화      253
2023    입산자실화      187
2024    입산자실화       46
Name: count, dtype: int64

#### 계절별 발생원인(기타) 산불 발생 건수

In [62]:
season_cause = df.groupby('계절')['발생원인_기타'].value_counts()
most_freq_cause = season_cause.groupby('계절').idxmax()
season_cause = season_cause[most_freq_cause]
season_cause

계절  발생원인_기타
가을  입산자실화      195
겨울  입산자실화      405
봄   입산자실화      977
여름  입산자실화      202
Name: count, dtype: int64

#### 월별 발생원인(기타) 산불 발생 건수

In [64]:
month_cause = df.groupby('발생일시_월')['발생원인_기타'].value_counts()
most_freq_cause = month_cause.groupby('발생일시_월').idxmax()
month_cause = month_cause[most_freq_cause]
month_cause

발생일시_월  발생원인_기타
1       입산자실화      114
2       입산자실화      211
3       쓰레기소각      419
4       입산자실화      429
5       입산자실화      242
6       입산자실화      142
7       입산자실화       18
8       입산자실화       42
9       입산자실화       38
10      입산자실화       71
11      입산자실화       86
12      입산자실화       80
Name: count, dtype: int64

#### 시간대별 발생원인(기타) 산불 발생 건수

In [66]:
time_cause = df.groupby('발생일시_시간')['발생원인_기타'].value_counts()
most_freq_cause = time_cause.groupby('발생일시_시간').idxmax()
time_cause = time_cause[most_freq_cause]
time_cause

발생일시_시간  발생원인_기타
0        입산자실화       21
1        입산자실화       12
2        입산자실화        7
3        입산자실화       17
4        원인미상        13
5        입산자실화       25
6        입산자실화       11
7        입산자실화       12
8        입산자실화       21
9        입산자실화       37
10       입산자실화       81
11       입산자실화      141
12       입산자실화      188
13       입산자실화      239
14       입산자실화      280
15       입산자실화      225
16       입산자실화      148
17       입산자실화       95
18       입산자실화       70
19       입산자실화       49
20       입산자실화       28
21       입산자실화       27
22       원인미상        19
23       입산자실화       22
Name: count, dtype: int64

#### 계절별 시간대 산불 발생 건수

In [67]:
df.head(1)

,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,진화종료시간_년,진화종료시간_월,진화종료시간_일,진화종료시간_시간,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_구분,발생원인_세부원인,발생원인_기타,피해면적_합계,계절,시간대
0,2024,12,31,14,6,2025,1,1,10,대구,대구,군위,삼국유사,화북,기,기타(직접입력),조사중,14.5,겨울,오후


In [70]:
cause_time = df.groupby('계절')['발생일시_시간'].value_counts()
most_time_freq = cause_time.groupby('계절').idxmax()
cause_time= cause_time[most_time_freq]
cause_time

계절  발생일시_시간
가을  14          70
겨울  14         228
봄   14         484
여름  15          65
Name: count, dtype: int64

In [83]:
df.head(1)

,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,진화종료시간_년,진화종료시간_월,진화종료시간_일,진화종료시간_시간,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_구분,발생원인_세부원인,발생원인_기타,피해면적_합계,계절,시간대
0,2024,12,31,14,6,2025,1,1,10,대구,대구,군위,삼국유사,화북,기,기타(직접입력),조사중,14.5,겨울,오후


---

In [199]:
del df['시간대']

In [200]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5455 entries, 0 to 5454
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   발생일시_년     5455 non-null   int64  
 1   발생일시_월     5455 non-null   int64  
 2   발생일시_일     5455 non-null   int64  
 3   발생일시_시간    5455 non-null   int64  
 4   발생일시_요일    5455 non-null   object 
 5   진화종료시간_년   5455 non-null   int64  
 6   진화종료시간_월   5455 non-null   int64  
 7   진화종료시간_일   5455 non-null   int64  
 8   진화종료시간_시간  5455 non-null   int64  
 9   발생장소_관서    5454 non-null   object 
 10  발생장소_시도    5455 non-null   object 
 11  발생장소_시군구   5432 non-null   object 
 12  발생장소_읍면    4919 non-null   object 
 13  발생장소_동리    4934 non-null   object 
 14  발생원인_구분    4514 non-null   object 
 15  발생원인_세부원인  5455 non-null   object 
 16  발생원인_기타    5455 non-null   object 
 17  피해면적_합계    5455 non-null   float64
 18  계절         5455 non-null   object 
dtypes: float64(1), int64(8), object(10)
memory usage

In [201]:
df_corr = df[['발생일시_년', '발생일시_월', '발생일시_일', '발생일시_시간', '발생일시_요일', '발생장소_관서', '발생장소_시도', '발생장소_시군구', '발생장소_읍면', '발생장소_동리', '발생원인_기타', '피해면적_합계', '계절']]
df_corr.head(2)

,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_기타,피해면적_합계,계절
0,2024,12,31,14,화,대구,대구,군위,삼국유사,화북,조사중,14.50,겨울
1,2024,12,30,15,월,경기,경기,양평,강하,왕창,재처리부주의,0.06,겨울


In [ ]:
encoder = LabelEncoder()
df['발생일시_요일'] = encoder.fit_transform(df['발생일시_요일'])
df['발생장소_관서'] = encoder.fit_transform(df['발생장소_관서'])
df['발생장소_시도'] = encoder.fit_transform(df['발생장소_시도'])
df['발생장소_시군구'] = encoder.fit_transform(df['발생장소_시군구'])
df['발생장소_읍면'] = encoder.fit_transform(df['발생장소_읍면'])
df['발생장소_동리'] = encoder.fit_transform(df['발생장소_동리'])
df['발생원인_기타'] = encoder.fit_transform(df['발생원인_기타'])
df['계절'] = encoder.fit_transform(df['계절'])
df['시간대'] = encoder.fit_transform(df['시간대'])

In [72]:
df.head(1)

,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,진화종료시간_년,진화종료시간_월,진화종료시간_일,진화종료시간_시간,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_구분,발생원인_세부원인,발생원인_기타,피해면적_합계,계절,시간대
0,2024,12,31,14,6,2025,1,1,10,대구,대구,군위,삼국유사,화북,기,기타(직접입력),조사중,14.5,겨울,오후


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5455 entries, 0 to 5454
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   발생일시_년     5455 non-null   int64   
 1   발생일시_월     5455 non-null   int64   
 2   발생일시_일     5455 non-null   int64   
 3   발생일시_시간    5455 non-null   int64   
 4   발생일시_요일    5455 non-null   int64   
 5   진화종료시간_년   5455 non-null   int64   
 6   진화종료시간_월   5455 non-null   int64   
 7   진화종료시간_일   5455 non-null   int64   
 8   진화종료시간_시간  5455 non-null   int64   
 9   발생장소_관서    5454 non-null   object  
 10  발생장소_시도    5455 non-null   object  
 11  발생장소_시군구   5432 non-null   object  
 12  발생장소_읍면    4919 non-null   object  
 13  발생장소_동리    4934 non-null   object  
 14  발생원인_구분    4514 non-null   object  
 15  발생원인_세부원인  5455 non-null   object  
 16  발생원인_기타    5455 non-null   object  
 17  피해면적_합계    5455 non-null   float64 
 18  계절         5455 non-null   object  
 19  시간대        5406 non-null   

In [204]:
df_corr = df[['발생일시_년', '발생일시_월', '발생일시_일', '발생일시_시간', '발생일시_요일', '발생장소_관서', '발생장소_시도', '발생장소_시군구', '발생장소_읍면', '발생장소_동리', '발생원인_기타', '피해면적_합계', '계절']]
df_corr.head(2)

,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_기타,피해면적_합계,계절
0,2024,12,31,14,6,대구,대구,군위,삼국유사,화북,조사중,14.50,겨울
1,2024,12,30,15,3,경기,경기,양평,강하,왕창,재처리부주의,0.06,겨울
